In [1]:
##Request the data and convert to a csv file

In [22]:
#import the following libraries
import requests
import csv
import json
import pandas as pd
import os.path

In [23]:
#you will need an API key, which you can get through https://www.census.gov/data/developers.html
api_key_input = input('Enter your API key: ')
census_api_key = api_key_input

In [24]:
#states dictionary

states = {'AL':'01', 'AK':'02', 'AZ':'04', 'AR':'05', 'CA':'06', 'CO':'08', 'CT':'09', 'DE':'10', 'DC':'11', 'FL':'12', 'GA':'13', 'HI':'15', 
            'ID':'16', 'IL':'17', 'IN':'18', 'IA':'19', 'KS':'20', 'KY':'21', 'LA':'22', 'ME':'23', 'MD':'24', 'MA':'25', 'MI':'26', 'MN':'27', 
            'MS':'28', 'MO':'29', 'MT':'30', 'NE':'31', 'NV':'32', 'NH':'33', 'NJ':'34', 'NM':'35', 'NY':'36', 'NC':'37', 'ND':'38', 'OH':'39', 
            'OK':'40', 'OR':'41', 'PA':'42', 'RI':'44', 'SC':'45', 'SD':'46', 'TN':'47', 'TX':'48', 'UT':'49', 'VT':'50', 'VA':'51', 'WA':'53', 
            'WV':'54', 'WI':'55', 'WY':'56', 'AS':'60', 'GU':'66', 'MP':'69', 'PR':'72', 'VI':'78'}
state_input = input('Enter a valid two letter abbreviation for the state or territory of your choice: ')

while state_input not in states:
    print('Abbreviation is invalid')
    state_input = input('Enter a valid two letter abbreviation for the state or territory of your choice: ')

state_fips = states[state_input]




In [25]:
#use the get() function to request the data from the API
#more information on formatting and access to datasets and variables can be found on the census developers webpage


response = requests.get("https://api.census.gov/data/2019/acs/acs5?get=NAME,GEO_ID,B19013_001E&for=block group:*&in=state:{}&in=county:*&in=tract:*&key={}".format(state_fips, census_api_key))

In [26]:
#run this to check if everything is running properly
#here is a good reference guide for all of the possible responses: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status
print(response.status_code)

200


In [31]:
#load the data as a json file and print to check if everything looks fine
result = json.loads(response.text)

print(result)

In [28]:
#use pandas to convert the json into csv files and save into directory
if os.path.isfile(r'income_bg_{}.csv'.format(state_input)):
    file_save_input = input('File already exists. Would you like to overwrite it?(y/n)')
    if file_save_input == 'y' or file_save_input == 'Y':
        print('overwriting file in directory')
        pd.DataFrame(result).to_csv(r'income_bg_{}.csv'.format(state_input))
    elif file_save_input == 'n' or file_save_input == 'N':
        pass
else:
    print('saving file to directory')
    pd.DataFrame(result).to_csv(r'income_bg_{}.csv'.format(state_input))
#if you want to remove the index from the csv file, simply add ", index = False" after the file directory within the parentheses

saving file to directory


In [29]:
df = pd.read_csv(r'income_bg_{}.csv'.format(state_input))

In [32]:
#clean the data to how you need it
df.head()

,Unnamed: 0,0,1,2,3,4,5,6
0,0,NAME,GEO_ID,B19013_001E,state,county,tract,block group
1,1,"Block Group 3, Census Tract 402, Botetourt Cou...",1500000US510230402003,49894,51,023,040200,3
2,2,"Block Group 2, Census Tract 402, Botetourt Cou...",1500000US510230402002,69038,51,023,040200,2
3,3,"Block Group 1, Census Tract 404.04, Virginia B...",1500000US518100404041,96029,51,810,040404,1
4,4,"Block Group 3, Census Tract 404.04, Virginia B...",1500000US518100404043,97130,51,810,040404,3


In [34]:
df.columns

Index(['Unnamed: 0', '0', '1', '2', '3', '4', '5', '6'], dtype='object')

In [35]:
#change column names
df.columns = ['index', 'NAME', 'GEO_ID', 'INCOME', 'state', 'county', 'tract', 'block_group']
df.head()

,index,NAME,GEO_ID,INCOME,state,county,tract,block_group
0,0,NAME,GEO_ID,B19013_001E,state,county,tract,block group
1,1,"Block Group 3, Census Tract 402, Botetourt Cou...",1500000US510230402003,49894,51,023,040200,3
2,2,"Block Group 2, Census Tract 402, Botetourt Cou...",1500000US510230402002,69038,51,023,040200,2
3,3,"Block Group 1, Census Tract 404.04, Virginia B...",1500000US518100404041,96029,51,810,040404,1
4,4,"Block Group 3, Census Tract 404.04, Virginia B...",1500000US518100404043,97130,51,810,040404,3


In [36]:
#splitting GEO_ID column to just get the value I need
GEO_ID = df['GEO_ID']
df[['US_Code', 'GEO_ID']] = GEO_ID.str.split("US", n=1, expand=True)

In [37]:
#delete new column that I don't need
del df['US_Code']
df.head()

,index,NAME,GEO_ID,INCOME,state,county,tract,block_group
0,0,NAME,None,B19013_001E,state,county,tract,block group
1,1,"Block Group 3, Census Tract 402, Botetourt Cou...",510230402003,49894,51,023,040200,3
2,2,"Block Group 2, Census Tract 402, Botetourt Cou...",510230402002,69038,51,023,040200,2
3,3,"Block Group 1, Census Tract 404.04, Virginia B...",518100404041,96029,51,810,040404,1
4,4,"Block Group 3, Census Tract 404.04, Virginia B...",518100404043,97130,51,810,040404,3


In [38]:
#drop row with same values as column heads
df = df.drop(df.index[0])
df

,index,NAME,GEO_ID,INCOME,state,county,tract,block_group
1,1,"Block Group 3, Census Tract 402, Botetourt Cou...",510230402003,49894,51,023,040200,3
2,2,"Block Group 2, Census Tract 402, Botetourt Cou...",510230402002,69038,51,023,040200,2
3,3,"Block Group 1, Census Tract 404.04, Virginia B...",518100404041,96029,51,810,040404,1
4,4,"Block Group 3, Census Tract 404.04, Virginia B...",518100404043,97130,51,810,040404,3
5,5,"Block Group 2, Census Tract 404.04, Virginia B...",518100404042,94875,51,810,040404,2
...,...,...,...,...,...,...,...,...
5328,5328,"Block Group 1, Census Tract 27, Norfolk city, ...",517100027001,60694,51,710,002700,1
5329,5329,"Block Group 4, Census Tract 29, Norfolk city, ...",517100029004,-666666666,51,710,002900,4
5330,5330,"Block Group 1, Census Tract 29, Norfolk city, ...",517100029001,22007,51,710,002900,1
5331,5331,"Block Group 1, Census Tract 8103, Petersburg c...",517308103001,51500,51,730,810300,1


In [39]:
#remove nulls
df = df[df.INCOME != '-666666666']
df

,index,NAME,GEO_ID,INCOME,state,county,tract,block_group
1,1,"Block Group 3, Census Tract 402, Botetourt Cou...",510230402003,49894,51,023,040200,3
2,2,"Block Group 2, Census Tract 402, Botetourt Cou...",510230402002,69038,51,023,040200,2
3,3,"Block Group 1, Census Tract 404.04, Virginia B...",518100404041,96029,51,810,040404,1
4,4,"Block Group 3, Census Tract 404.04, Virginia B...",518100404043,97130,51,810,040404,3
5,5,"Block Group 2, Census Tract 404.04, Virginia B...",518100404042,94875,51,810,040404,2
...,...,...,...,...,...,...,...,...
5327,5327,"Block Group 1, Census Tract 22, Norfolk city, ...",517100022001,120385,51,710,002200,1
5328,5328,"Block Group 1, Census Tract 27, Norfolk city, ...",517100027001,60694,51,710,002700,1
5330,5330,"Block Group 1, Census Tract 29, Norfolk city, ...",517100029001,22007,51,710,002900,1
5331,5331,"Block Group 1, Census Tract 8103, Petersburg c...",517308103001,51500,51,730,810300,1


In [43]:
#save final copy
save_new = input('Would you like to save a separate copy of your cleaned csv (1), or would you like to overwrite your raw csv (2)? ')
while save_new == '1' or save_new == '0':
    save_new = input('Would you like to save a separate copy of your cleaned csv (1), or would you like to overwrite your raw csv (2)? ')
    if save_new == '1':
        os.path.isfile(r'income_bg_{}_final.csv'.format(state_input)) 
        print('saving file to directory')
        break
    elif save_new == '2':
        pd.DataFrame(df).to_csv(r'income_bg_{}.csv'.format(state_input))
        print('overwriting old file with new file')
        break
    

saving file to directory
